In [13]:
import re  # Importiert das re Modul für reguläre Ausdrücke.
import hashlib  # Importiert das hashlib Modul für Hash-Funktionen.
import numpy as np  # Importiert das numpy Modul für numerische Operationen.

def preprocess_text(text):
    """ Entfernt Sonderzeichen und führt eine einfache Textbereinigung durch. """
    # Ersetzt Sonderzeichen durch nichts und konvertiert den Text in Kleinbuchstaben.
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

def create_shingles(text, k=3):
    """ Erstellt Shingles (n-Gramme) der Länge k aus dem Text. """
    words = text.split()  # Zerlegt den Text in Wörter.
    # Erzeugt Shingles der Länge k aus dem Text.
    for i in range(len(words) - k + 1):
        yield ' '.join(words[i:i + k])

def minhash(shingles, num_perm=256):
    """ Erstellt eine MinHash-Signatur für ein Set von Shingles. """
    # Initialisiert ein Array mit 'inf' für jede der num_perm Permutationen.
    signatures = np.full(num_perm, float('inf'))
    # Durchläuft jedes Shingle im Set.
    for shingle in shingles:
        # Berechnet Hash-Werte für jedes Shingle mit num_perm verschiedenen Salts.
        for i in range(num_perm):
            hash_val = int(hashlib.sha256(f'{shingle}{i}'.encode()).hexdigest(), 16) % (2**32 - 1)
            # Speichert den kleinsten Hash-Wert für jede Permutation.
            signatures[i] = min(signatures[i], hash_val)
    return signatures

def jaccard_similarity(set1, set2):
    """ Berechnet die Jaccard-Ähnlichkeit zwischen zwei MinHash-Signaturen. """
    # Zählt die Anzahl der übereinstimmenden Hash-Werte und teilt sie durch die Gesamtanzahl.
    return np.sum(set1 == set2) / len(set1)

# Lese die Datei und erstelle MinHash für jeden Artikel
file_path = '/Users/furkan/Downloads/articles-2.txt'  # Pfad zur Datei.
articles_minhash = {}  # Dictionary zur Speicherung der MinHash-Signaturen der Artikel.

# Öffnet die Datei und liest ihren Inhalt.
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.readlines()

# Durchläuft jede Zeile in der Datei.
for line in content:
    parts = line.split(' ', 1)  # Trennt die Artikel-ID vom Text.
    if len(parts) == 2:
        article_id, article_text = parts[0], parts[1]
        # Bereitet den Text vor und erzeugt Shingles.
        processed_text = preprocess_text(article_text)
        shingles = set(create_shingles(processed_text))
        # Berechnet die MinHash-Signatur für die Shingles.
        minhash_signature = minhash(shingles)
        # Speichert die Signatur im Dictionary.
        articles_minhash[article_id] = minhash_signature

# Suche nach ähnlichen Artikel-Paaren und berechne die Jaccard-Ähnlichkeit
plagiarism_candidates = []  # Liste zur Speicherung von Plagiat-Kandidaten.
article_ids = list(articles_minhash.keys())  # Liste der Artikel-IDs.

# Durchläuft jedes mögliche Paar von Artikeln.
for i in range(len(article_ids)):
    for j in range(i + 1, len(article_ids)):
        id1, id2 = article_ids[i], article_ids[j]
        # Berechnet die Jaccard-Ähnlichkeit der MinHash-Signaturen.
        similarity = jaccard_similarity(articles_minhash[id1], articles_minhash[id2])
        if similarity > 0.8:  # Überprüft, ob die Ähnlichkeit über dem Schwellenwert liegt.
            # Fügt das Paar und seine Ähnlichkeit zur Liste hinzu.
            plagiarism_candidates.append((id1, id2, similarity))

# Gibt die Ergebnisse aus.
for pair in plagiarism_candidates:
    print(f"Artikel {pair[0]} und {pair[1]} ähneln sich mit einer Ähnlichkeit von {pair[2]*100:.2f}%.")


Artikel t120 und t121 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t120 und t125 ähneln sich mit einer Ähnlichkeit von 6.25%.
Artikel t120 und t126 ähneln sich mit einer Ähnlichkeit von 6.64%.
Artikel t120 und t579 ähneln sich mit einer Ähnlichkeit von 1.56%.
Artikel t120 und t2186 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t120 und t3170 ähneln sich mit einer Ähnlichkeit von 1.56%.
Artikel t120 und t3582 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t120 und t4001 ähneln sich mit einer Ähnlichkeit von 1.56%.
Artikel t120 und t4210 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t120 und t4491 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t120 und t6247 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t121 und t145 ähneln sich mit einer Ähnlichkeit von 3.12%.
Artikel t121 und t318 ähneln sich mit einer Ähnlichkeit von 3.91%.
Artikel t121 und t559 ähneln sich mit einer Ähnlichkeit von 1.17%.
Artikel t121 und t1068 ähneln sich mit einer Ähnlichkei